In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, precision_score, recall_score
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, load_metric, Value

In [ ]:
# Load dataset
df = pd.read_excel('TheHackerNews_Dataset.xlsx')

In [ ]:
# Fill in NaN values with an empty string
df = df.fillna('')

In [ ]:
# Preprocess data
df['Text'] = df.Title + ". " + df.Article
df = df.drop(['Article','Title','Link'], axis=1)
df['Label'] = df['Label'].map({'Cyber_Attack':0, 'Malware':1, 'Vulnerability':2, 'Data_Breaches':3})

In [ ]:
# Split dataset into train and eval sets
train_df = df.sample(frac=0.8, random_state=42)
eval_df = df.drop(train_df.index)

In [ ]:
# Define GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='right')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Define tokenization function using FastTokenize feature of datasets library
def tokenize_function(examples):
    return tokenizer(examples['Text'], truncation=True, max_length=512)

In [ ]:
# Load data into datasets library format
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [ ]:
# Tokenize data
train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4)
eval_dataset = eval_dataset.map(tokenize_function, batched=True, num_proc=4)

In [ ]:
# Set the format of the datasets to PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Label'])

In [ ]:
# Define GPT-2 model
class GPT2Classifier(torch.nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.gpt2 = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=num_labels)
    def forward(self, input_ids, attention_mask, labels):
        output = self.gpt2(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

In [ ]:
# Instantiate GPT-2 model
model = GPT2Classifier(num_labels=4)

In [ ]:
# Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    mcc = matthews_corrcoef(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'mcc': mcc,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy='epoch',  # Updated save strategy
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
)

In [ ]:
# Instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Train
trainer.train()